### **1. 位置编码（Positional Encoding）**
---

#### **问题背景**
Transformer 模型不同于 RNN，不具备处理序列顺序的自然能力。它将输入序列视为一个整体并进行并行处理，因此需要一种机制来引入序列的位置信息，以帮助模型理解输入数据的顺序。

#### **位置编码的定义**
位置编码（Positional Encoding）是 Transformer 解决序列顺序问题的一种方法，它在输入序列的词向量上加上特定的编码，以提供位置信息。

#### **常见的实现方式**
1. **正弦和余弦位置编码（Sine & Cosine Positional Encoding）**  
   - 在 Transformer 论文《Attention Is All You Need》中提出的方式。
   - 位置编码的计算公式如下：
     $$
     PE_{(pos, 2i)} = \sin(\frac{pos}{10000^{2i/d_{model}}})
     $$
     $$
     PE_{(pos, 2i+1)} = \cos(\frac{pos}{10000^{2i/d_{model}}})
     $$
   - 其中：
     - \( pos \) 是单词在序列中的位置。
     - \( i \) 是编码维度的索引。
     - \( d_{model} \) 是嵌入向量的维度。

   代码示例（PyTorch）：
   ```python
   import torch
   import math

   def positional_encoding(seq_len, d_model):
       pos = torch.arange(seq_len).unsqueeze(1)
       i = torch.arange(d_model).unsqueeze(0)
       angle_rates = 1 / torch.pow(10000, (2 * (i // 2)) / d_model)
       pe = pos * angle_rates
       pe[:, 0::2] = torch.sin(pe[:, 0::2])
       pe[:, 1::2] = torch.cos(pe[:, 1::2])
       return pe.unsqueeze(0)  # shape: (1, seq_len, d_model)

   encoding = positional_encoding(50, 512)
   print(encoding.shape)  # torch.Size([1, 50, 512])
   ```

2. **可学习的位置编码（Learnable Positional Encoding）**
   - 作为模型参数进行学习，而非使用固定的正弦/余弦函数。
   - 直接使用嵌入层（`nn.Embedding`）学习每个位置的权重。

   ```python
   import torch.nn as nn

   class LearnablePositionalEncoding(nn.Module):
       def __init__(self, max_len, d_model):
           super().__init__()
           self.encoding = nn.Embedding(max_len, d_model)

       def forward(self, x):
           pos_ids = torch.arange(x.shape[1]).unsqueeze(0).to(x.device)
           return self.encoding(pos_ids)

   pos_enc = LearnablePositionalEncoding(100, 512)
   print(pos_enc(torch.zeros(1, 10).long()).shape)  # torch.Size([1, 10, 512])
   ```

#### **作用**
- 在 Transformer 中，位置编码会与输入的词嵌入（word embeddings）相加，以提供位置信息，帮助模型捕获序列顺序。
- 由于正弦和余弦编码在维度上具有周期性，它可以在长序列上进行泛化。


### **2. 多头注意力机制（Multi-Head Attention）**
---

#### **问题背景**
传统的注意力机制只能关注输入序列的某些部分，而 Transformer 使用 **多头注意力机制** 来提高模型的表示能力，增强不同部分之间的依赖关系。

#### **多头注意力的定义**
多头注意力机制的核心思想是：  
将输入数据投影到多个不同的表示子空间，在这些子空间中分别计算注意力，然后将多个注意力结果合并。

#### **计算步骤**
1. **输入：**
   - 词嵌入矩阵 \( X \in \mathbb{R}^{seq\_len \times d\_model} \)

2. **步骤：**
   - 将输入分别线性映射为：
     - 查询矩阵 \( Q \)
     - 键矩阵 \( K \)
     - 值矩阵 \( V \)

   $$ Q = XW_Q, \quad K = XW_K, \quad V = XW_V $$

   - 计算注意力权重：
     $$ Attention(Q, K, V) = \text{softmax} \left(\frac{QK^T}{\sqrt{d_k}}\right) V $$

   - 将多个注意力头的输出拼接，投影到最终输出维度。

#### **多头注意力的优点**
1. **增强表达能力：** 每个注意力头可以关注不同的语义信息（如短程关系、长程依赖）。
2. **并行化处理：** 多个头同时运行，提高计算效率。
3. **更好的特征捕获：** 避免单头注意力局限于特定维度。

#### **数学公式**
对于第 \( i \) 个注意力头：
$$
head_i = \text{softmax}\left(\frac{Q_i K_i^T}{\sqrt{d_k}}\right) V_i
$$
最终输出为：
$$
MultiHead(Q, K, V) = \text{Concat}(head_1, head_2, ..., head_h) W^O
$$

#### **代码示例（PyTorch）**
```python
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0  # 确保可以整除
        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_out = nn.Linear(d_model, d_model)

    def forward(self, query, key, value):
        batch_size = query.size(0)

        # 线性变换并分头 (batch_size, seq_len, d_model) -> (batch_size, num_heads, seq_len, d_k)
        Q = self.w_q(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.w_k(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.w_v(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # 计算注意力权重
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attention_weights = torch.nn.functional.softmax(attention_scores, dim=-1)

        # 加权求和
        output = torch.matmul(attention_weights, V)
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)

        return self.w_out(output)

# 使用示例
mha = MultiHeadAttention(512, 8)
x = torch.rand(1, 10, 512)  # (batch_size, seq_len, d_model)
out = mha(x, x, x)
print(out.shape)  # torch.Size([1, 10, 512])
```

---

### **总结**
- **位置编码**：为输入序列提供位置信息，常见方法有正弦/余弦编码或可学习编码。
- **多头注意力**：将输入投影到多个子空间，计算不同语义的注意力，提升模型的表达能力。

如果你有任何具体问题或想要深入了解某个部分，欢迎继续讨论！